In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import os
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, BooleanType
from pyspark.sql.functions import collect_list, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import first
import pandas as pd

# Сначала подготовим датасет с последовательностями конверсий

In [26]:
def clean_petco_conversions(data_logs):
  data_conversions = data_logs.filter(data_logs.action == 'conversion')

  # Оставляем только нужные столбцы
  data_conversions_filtered = data_conversions.select('customer_user_id', 'customer_id', 'timestamp', 'revenue')

  # Добавляем новый столбец 'conversion' со значением 1
  data_conversions_filtered = data_conversions_filtered.withColumn('conversion', F.lit(1))

  # Переставляем столбцы, чтобы 'conversion' был на третьем месте
  data_conversions_filtered = data_conversions_filtered.select(
      'customer_user_id', 'customer_id', 'conversion', 'timestamp', 'revenue'
  )

  data_conversions = data_conversions_filtered.orderBy(['customer_user_id', 'timestamp'])
  data_conversions = data_conversions.withColumnRenamed('customer_user_id', 'user_id') \
                    .withColumnRenamed('customer_id', 'item_id')

  return data_conversions.dropna()

In [25]:
def save_dataset_parquet(dataset, folder_path, file_name):
  # Проверяем, существует ли папка, если нет - создаем
  if not os.path.exists(folder_path):
      os.makedirs(folder_path)
      print(f"Папка {folder_path} успешно создана!")
  else:
      print(f"Папка {folder_path} уже существует.")

  file_path = folder_path+file_name

  # Сохранение в формате Parquet
  dataset.write.parquet(file_path, mode="overwrite")
  print(f"Датасет сохранён по пути {file_path}.")

In [7]:
def actions_to_sasrec_format(data_conversions_cleaned):
  data_conversions_processed = data_conversions_cleaned.withColumn(
      "datetime", F.date_format("timestamp", "yyyy-MM-dd HH:mm:ss")  # Преобразуем timestamp в строку с датой и временем
  ).withColumn(
      "weight", F.lit(1)  # Добавляем колонку weight со значением 1
  ).select("user_id", "item_id", "datetime", "weight")  # Оставляем нужные колонки

  data_conversions_processed = data_conversions_processed \
    .withColumn("user_id", col("user_id").cast("int")) \
    .withColumn("item_id", col("item_id").cast("int")) \
    .withColumn("datetime", col("datetime").cast("timestamp")) \
    .withColumn("weight", col("weight").cast("int"))

  print(data_conversions_processed.dtypes)

  data_conversions_processed = data_conversions_processed.toPandas().dropna()
  data_conversions_processed['user_id'] = data_conversions_processed['user_id'].apply(lambda x: int(x))

  return data_conversions_processed

In [11]:
def save_dataset_pickle(dataset, folder_path, file_name):
  # Проверяем, существует ли папка, если нет - создаем
  if not os.path.exists(folder_path):
      os.makedirs(folder_path)
      print(f"Папка {folder_path} успешно создана!")
  else:
      print(f"Папка {folder_path} уже существует.")

  file_path = folder_path+file_name

  # Сохранение в формате Parquet
  dataset.to_pickle(file_path)
  print(f"Датасет сохранён по пути {file_path}.")

# Теперь подготовим датасет с описанием товаров

In [15]:
def clean_petco_items(data_items):
  # Оставляем только столбцы id, customer_id, name, name_lower, metadata_json
  data_items_filtered = data_items.select("id", "customer_id", "name", "name_lower", "metadata_json")

  # Определим структуру для metadata_json
  metadata_schema = create_sctructure_metadata()

  # Преобразуем metadata_json в структуру
  data_items_parsed = data_items_filtered.withColumn(
      "metadata_parsed", from_json(col("metadata_json"), metadata_schema)
  )

  # Извлекаем нужные атрибуты из metadata_parsed в отдельные столбцы
  data_items_with_columns = data_items_parsed.select(
      "id", "customer_id", "name", "name_lower",
      "metadata_parsed.url", "metadata_parsed.image_url", "metadata_parsed.itemname",
      "metadata_parsed.facets", "metadata_parsed.group_ids",
      "metadata_parsed.PTC_OMNI_REPEAT_DELIVERY_FL", "metadata_parsed.PTC_OMNI_PRIMARY_ITEM_FLAG",
      "metadata_parsed.PTC_OMNI_TAXONOMY", "metadata_parsed.PTC_OMNI_PERSONALIZED_ITEM_FL",
      "metadata_parsed.PTC_OMNI_SAME_DAY_DELIVERY_FG", "metadata_parsed.PTC_OMNI_PDP_BEHAVIOR_TEMPLATE",
      "metadata_parsed.PTC_OMNI_BOPUS_FLAG", "metadata_parsed.PTC_OMNI_BRAND_PRIMARY",
      "metadata_parsed.itemurl", "metadata_parsed.itemimg", "metadata_parsed.UPC_NUMBER"
  )

  same_columns = ['name', 'name_lower', 'url', 'image_url', 'itemname', 'facets']

  # Регистрируем UDF для вычисления пересечения
  intersection_udf = udf(array_intersection, ArrayType(StringType()))

  # Группируем по customer_id и собираем все group_ids в список списков, затем вычисляем пересечение
  data_items_intersect = data_items_with_columns.groupBy("customer_id") \
      .agg(collect_list("group_ids").alias("group_ids_list")) \
      .withColumn("group_ids_intersect", intersection_udf("group_ids_list"))

  # Агрегируем data_items_with_columns по customer_id:
  # Для каждого столбца из same_columns (кроме customer_id) берем первое значение
  aggregated_same = data_items_with_columns.groupBy("customer_id").agg(
      *[first(col_name).alias(col_name) for col_name in same_columns if col_name != "customer_id"]
  )

  # Теперь объединяем агрегированный DataFrame с DataFrame, содержащим group_ids_intersect.
  # Предполагается, что data_items_intersect был получен ранее, например:
  final_df = aggregated_same.join(
      data_items_intersect.select("customer_id", "group_ids_intersect"),
      on="customer_id",
      how="left"
  )

  final_df = final_df.withColumn("How_to_get_it", col("facets.`How to get it`")) \
                   .withColumn("Primary_Brand", col("facets.`Primary Brand`")) \
                   .withColumn("Primary_Color", col("facets.`Primary Color`")) \
                   .withColumn("Killed_Item_flag", col("facets.`Killed Item flag`")) \
                   .withColumn("Primary_Pet_Type", col("facets.`Primary Pet Type`")) \
                   .withColumn("Personalized_Item_flag", col("facets.`Personalized Item flag`")) \
                   .withColumn("Prescription_Medication", col("facets.`Prescription Medication`"))

  # Если исходный столбец facets больше не нужен, можно его удалить:
  final_df = final_df.drop("facets")

  return final_df

In [16]:
def create_sctructure_metadata():
  metadata_schema = StructType([
      StructField("url", StringType(), True),
      StructField("image_url", StringType(), True),
      StructField("facets", StructType([
          StructField("How to get it", ArrayType(StringType()), True),
          StructField("Primary Brand", ArrayType(StringType()), True),
          StructField("Primary Color", ArrayType(StringType()), True),
          StructField("Killed Item flag", ArrayType(StringType()), True),
          StructField("Primary Pet Type", ArrayType(StringType()), True),
          StructField("Personalized Item flag", ArrayType(StringType()), True),
          StructField("Prescription Medication", ArrayType(StringType()), True)
      ]), True),
      StructField("group_ids", ArrayType(StringType()), True),
      StructField("PTC_OMNI_REPEAT_DELIVERY_FL", StringType(), True),
      StructField("parentCatEntryIDAsString", StringType(), True),
      StructField("PTC_OMNI_PRIMARY_ITEM_FLAG", StringType(), True),
      StructField("mfName", StringType(), True),
      StructField("PTC_OMNI_IN_STORE_ONLY_FLAG", StringType(), True),
      StructField("PTC_OMNI_PROP_65_FLAG", StringType(), True),
      StructField("PTC_OMNI_TAXONOMY", StringType(), True),
      StructField("PTC_OMNI_PERSONALIZED_ITEM_FL", StringType(), True),
      StructField("parentCatEntryID", StringType(), True),
      StructField("startDate", StringType(), True),
      StructField("deactivated", BooleanType(), True),
      StructField("PTC_OMNI_SAME_DAY_DELIVERY_FG", StringType(), True),
      StructField("PTC_OMNI_PDP_BEHAVIOR_TEMPLATE", StringType(), True),
      StructField("itemname", StringType(), True),
      StructField("PTC_OMNI_BOPUS_FLAG", StringType(), True),
      StructField("PTC_OMNI_BRAND_PRIMARY", StringType(), True),
      StructField("itemurl", StringType(), True),
      StructField("itemimg", StringType(), True),
      StructField("UPC_NUMBER", StringType(), True)
  ])

  return metadata_schema

In [18]:
# Функция для вычисления пересечения списка списков
def array_intersection(arrays):
    if not arrays:
        return []
    # Инициализируем пересечение первым списком
    result = set(arrays[0])
    # Пересекаем с каждым следующим списком
    for arr in arrays[1:]:
        result = result.intersection(arr)
    return list(result)

In [1]:
def items_to_sasrec_format(data_items_cleaned):
  data_items_cleaned = data_items_cleaned.select('customer_id', 'group_ids_intersect', 'How_to_get_it', 'Primary_Brand', 'Primary_Pet_Type')

  # Разворачиваем списки и добавляем новые колонки с признаками
  df_category = data_items_cleaned.select(
      "customer_id",
      F.explode("group_ids_intersect").alias("value")
  ).withColumn("feature", F.lit("Category")).withColumnRenamed("customer_id", "id")

  df_delivery = data_items_cleaned.select(
      "customer_id",
      F.explode("How_to_get_it").alias("value")
  ).withColumn("feature", F.lit("Delivery")).withColumnRenamed("customer_id", "id")

  df_brand = data_items_cleaned.select(
      "customer_id",
      F.explode("Primary_Brand").alias("value")
  ).withColumn("feature", F.lit("Brand")).withColumnRenamed("customer_id", "id")

  df_pet = data_items_cleaned.select(
      "customer_id",
      F.explode("Primary_Pet_Type").alias("value")
  ).withColumn("feature", F.lit("Pet")).withColumnRenamed("customer_id", "id")

  # Объединяем все эти датафреймы в один
  data_items_processed = df_category.union(df_delivery).union(df_brand).union(df_pet)
  data_items_processed = data_items_processed.toPandas().dropna()
  data_items_processed['id'] = data_items_processed['id'].apply(lambda x: int(x))

  return data_items_processed